In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 10, 10
from matplotlib.pyplot import imshow
from numpy.linalg import norm
import rawpy
import PIL
from uncertainties import unumpy as unp
from uncertainties import ufloat
from uncertainties.umath import *

#dummyvalues
r_mond=1737000. #r_mond in meter
pixerr=10

In [2]:
#calculating the norm for arrays consisting of ufloat
def normunc(x):
    y=x**2
    #return unumpy.sqrt(sum(y))
    return sqrt(sum(y))

def arccosunc(x):
    yn=np.arccos(x.n)
    ys=np.abs(1/np.sqrt(1-x.n**2)*x.s)
    return ufloat(yn, ys)

def arcsinunc(x):
    yn=np.arcsin(x.n)
    ys=np.abs(1/np.sqrt(1-x.n**2)*x.s)
    return ufloat(yn, ys)

Function to find the radius of the Moon in pixels via Sekanten-Verfahren

In [4]:
R= lambda theta: np.array([[np.cos(theta), -np.sin(theta)],
                          [np.sin(theta), np.cos(theta)]])

def mittelsenkrechte(XX,YY):
    '''
    Input: XX, YY: x- und y-coordinates of the two points
    Output: center point of the center and a vector along the perpendicular bisector (Mittelsenkrechte)
    #Output: The funktion which describes the perpendicular bisector as f(t)=vec1+t*vec2
    '''
    R90=R(np.pi*.5)
    koord1=np.array([XX[0],YY[0]])
    koord2=np.array([XX[1], YY[1]])
    mitte=0.5*(koord1+koord2)
    verbindung=koord2-koord1
    mitte_vec=np.dot(R90,verbindung)
    mitte_vec/=normunc(mitte_vec)
    return mitte, mitte_vec
    #return lambda t: mitte+t*mitte_vec

def schnittpunkt(m1, v1, m2, v2):
    #Finds t & s for the intersection point of the two lines m1+t*v1 und m2+s*v2
    m=m1-m2
    A=np.array([-v1,v2]).T
    return np.linalg.solve(A,m)

def schnittpunktunc(m1, v1, m2, v2):
    #Finds t & s for the intersection point of the two lines m1+t*v1 und m2+s*v2 with uncertainties
    m=m1-m2
    A=np.array([-v1,v2]).T
    K,G=m[0],m[1]
    a,b=A[0,0],A[0,1]
    c,d=A[1,0],A[1,1]
    x=G/(c-d*a/b)
    y=(K-a*x)/b
    return np.array([x,y])

def radius_finder(XX_werte, YY_werte):
    '''
    This function finds the radius of the Moon in Pixels via Sekanten
    Input: X_values, Y_values: x- and y- coordinates of the Sekanten endpoints
    Output: radius: Radius of the Moon in pixels
            circle center: The middle point of the Moon
    '''
    radius=0
    nx=len(XX_werte)
    if nx!=len(YY_werte):
        print('Mistake')
        return 0
    ax=[]; ay=[]
    for i in range(nx):
        ax.append(ufloat(XX_werte[i], pixerr))
        ay.append(ufloat(YY_werte[i], pixerr))
    X_werte=np.array(ax)
    Y_werte=np.array(ay)
    mitte1,vec1=mittelsenkrechte(X_werte[:2], Y_werte[:2])
    mitte2,vec2=mittelsenkrechte(X_werte[round(nx*0.5-0.2):], Y_werte[round(nx*0.5-0.2):])
    #t=schnittpunkt(mitte1, vec1, mitte2, vec2)
    t=schnittpunktunc(mitte1, vec1, mitte2, vec2)
    kreismittelpunkt=(mitte1+t[0]*vec1+mitte2+t[1]*vec2)*0.5
    for i in range(nx):
        radius+=normunc(np.array([X_werte[i], Y_werte[i]])-kreismittelpunkt)
    return radius/nx, kreismittelpunkt

We define a coordinate system. The following function calculates the coordinates of any point you have selected in the overview image.

In [5]:
'''
    Calculation of the coordinate system:
    Our coordinate system is defined by the connection line of the two shadow ends and their perpendicular bisector.
    pixerr : The error which occurs through the selection of the points by hand.
'''
XY1=np.array([ufloat(1539.35, pixerr), ufloat(5752.34, pixerr)])   #The endpoint of the shadow line below
XY2=np.array([ufloat(3699.66, pixerr), ufloat(509.084, pixerr)])    #The endpoint of the shadow line above

origin = .5*(XY1+XY2)
a_y=origin
b_y=(XY2-XY1)
a_x=origin
b_x=np.array([XY1[1]-XY2[1], XY2[0]-XY1[0]])

x_axis = lambda t: a_x + t*b_x    # x-axis (Senkrechte auf Verbindungsgerade)
y_axis = lambda t: a_y + t*b_y    # y-axis (Verbindungsgerade)


def coordinates(x,y):
    
    '''
        Input: pixel-coordinates of the marked point
        Output: coordinates of the point in our coordinate system
    '''

    p = np.array([ufloat(x, pixerr), ufloat(y, pixerr)])

    t = (b_y[1]*(p[0]-a_x[0]) + b_y[0]*(a_x[1]-p[1]))/(b_x[0]*b_y[1] - b_x[1]*b_y[0])
    t_tilde = (a_x[1] + t*b_x[1] - p[1])/b_y[1]

    x_coord = x_axis(t) - a_x
    x_coord = (x_coord[0]**2 + x_coord[1]**2)**(.5)
    if unp.nominal_values(t)<0:
        x_coord=-x_coord

    y_coord = p-x_axis(t)
    y_coord = (y_coord[0]**2 + y_coord[1]**2)**(.5)
    if unp.nominal_values(t_tilde)>0:
        y_coord = -y_coord

    return x_coord, y_coord

The following box calculates Beta-Doppelstrich (angle between y-axis and shadow line). Beta = Beta-Strich + Beta-Doppelstrich

In [6]:
''' 
    The reference point on the shadow line was choosen to be at (-779.5894216038151, 1.8338318595135774)
'''
x=abs(-779.5894216038151)    # Distance Origin-shadowborder at the hight of the x-axis
r_moon=2935.0    # calculated with radius finder, the value has an uncertainty of +/-9
beta_const = 2*np.arcsin(x/(2*r_moon))
print('rad: ',beta_const)
print('deg: ',360/(2*np.pi)*beta_const)

rad:  0.266405305489
deg:  15.2638996444


We calculate Beta from Beta-Strich and Beta-Doppelstrich. Where Beta-Strich can be found on the figures in the report. 

In [7]:
beta_strichstrich = beta_const
def betafinder(crater_bild, Rad_pixel_jpg=2935.0):
    '''
        Finds beta-strich from the figure and the hight in meters (H_mess) of the crater
        crater_bild are the coordinates of the crater
    '''
    hoehe=crater_bild[1]; breite=crater_bild[0] 
    r_strich=sqrt(Rad_pixel_jpg**2-hoehe**2)
    val=breite/r_strich
    if np.abs(val)>=0.999: val=val/abs(val)
    beta_strich=arcsinunc(val)
    return beta_strich+beta_strichstrich, hoehe*r_mond/Rad_pixel_jpg

We get 4 points (first two for the diameter and the other two for the shadow) and calculate with those informations the real diameter and depth of the crater.

In [9]:
def infos_crater_unc_ver2(XX_init,YY_init, beta, Phi, H_mess, Rad_pixel):
    pixeltometer = r_mond/Rad_pixel
    #H_mess must have an uncertainty!!
    XX=np.array([ufloat(XX_init[0], pixerr), ufloat(XX_init[1], pixerr), ufloat(XX_init[2], pixerr), ufloat(XX_init[3], pixerr)])
    YY=np.array([ufloat(YY_init[0], pixerr), ufloat(YY_init[1], pixerr), ufloat(YY_init[2], pixerr), ufloat(YY_init[3], pixerr)])
    coord=np.array([XX,YY])
    s_mess=normunc(coord[:,3]-coord[:,2])*pixeltometer
    D_mess=normunc(coord[:,1]-coord[:,0])*pixeltometer
    theta=arccosunc(H_mess/r_mond)#small phi in report
    D_real=D_mess/sin(theta)
    h=s_mess/sin(Phi)*sin(beta)
    return D_real, h

X=np.array([1,0,-1,3])
Y=np.array([0,1,0,3])
